# Phuture's Liquidity Mining Calculations - How much should we distribute?



First we import the coingecko API to retrieve the PHTR price, set the token0 and token1 balances of the PHTR/USDC liquidity pool(based on current prices) and set up our variables.

In [252]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

x=1000 #PHTR
y=120  #USDC
k=x*y
target_investment_size = 25000 #Denominated in USDC
target_price_impact = -0.5 #Adjust to target a different price impact. Insert as a percentage without the % sign
phtr_price = y/x #USD denominated PHTR price
usdc_price = 1
target_apr = 0.10 #Target APY for liquidity mining program once pool value required to satisfy price impact criterion has been reached
print(phtr_price)

0.12


Define a function to calculate the spot price for an infinitesimally small trade size.

In [253]:
def spot_rate(x,y):
    return x/y
print(spot_rate(x,y))
print (x,y)

8.333333333333334
1000 120


Define a function to calculate the actual trade price based on the target_investment_size.

In [254]:
def trade_price(x, y):
    new_y = target_investment_size + y
    new_x = (x*y)/new_y
    x_out = x-new_x
    return x_out/target_investment_size

Define a function to calculate the amount of price impact in percentatage terms generated from a trade of size equal to the target_investment_size. This function will be used in the while loop so we will prefix it with an underscore to delineate it as internal.

In [255]:
def _price_impact(x, y):
    abs = spot_rate(x,y) - trade_price(x,y)
    percent = (trade_price(x,y)/spot_rate(x,y) -1)*100
    return percent

Define an external price impact function with print statements.

In [256]:
def price_impact(x,y):
    abs = spot_rate(x,y) - trade_price(x,y)
    percent = (trade_price(x,y)/spot_rate(x,y) -1)*100
    print('Spot Price:',spot_rate(x,y),'USDC per 1 PHTR')
    print('Trade Price:',trade_price(x,y),'USDC per 1 PHTR')
    print('Price Impact: '+str(percent) +'%')
    return percent
print(price_impact(207333.33333333916,24880))

Spot Price: 8.333333333333568 USDC per 1 PHTR
Trade Price: 4.156642608928212 USDC per 1 PHTR
Price Impact: -50.12028869286287%
-50.12028869286287


Define a function that allows us to change the price of PHTR. This helps us to understand how the required amount of PHTR rewards change depending on the token price. 

In [257]:
def adjust_price(k,new_price=y/x):
    global x
    global y
    global phtr_price
    x= (k*new_price)**(1/2)
    y = (k/new_price)**(1/2)
    phtr_price = y/x
    return x/y

We then adjust the price to 0.4USD. This is to factor in some growth in the price of PHTR. The accuracy of this price is only some what relevant. The LM program is split into two 6 month tranches, so if the price is lagging this estimate we can bolster rewards in the second 6 month tranche.

In [258]:
print(adjust_price(k,2.25)) #Insert a new PHTR price to see how this affects the required PHTR to satisfy the target yield. The default is the current price.

2.25


Create a while loop that increments the balance of USDC and PHTR in the pool until the minimum price impact requirement is met. Here we are using our variable target_price_impact and as mentioned before, price impact is calculated on our target_investment_size of 25000 USDC. 

In [259]:
while _price_impact(x,y) < target_price_impact:
    x+=(spot_rate(x,y)*10)
    y+=10

After the while loop has completed we can calculate the USD balance of the pool, then based off of this, calculate the required USD to satisfy our target_apr and finally calculate the amount of PHTR that needs to be distributed. 

In [260]:
total_usd_value = x*phtr_price+y*usdc_price
usd_required = total_usd_value * target_apr
phtr_required = usd_required/phtr_price
print(phtr_price)

0.4444444444444444


Lastly, the print statements that show the pertinent output information.

In [261]:
print('Reserves: x:',x,'y:',y)
price_impact(x,y)
print('PHTR Price:',phtr_price,'USD')
print('Total USD value of pool required to satisfy target_price_impact criterion:',total_usd_value)
print('Required USD to satisfy target yield:',usd_required)
print('Required PHTR to satisfy target yield:',phtr_required)

Reserves: x: 11193752.11524227 y: 4975000.940107675
Spot Price: 2.2500000000000004 USDC per 1 PHTR
Trade Price: 2.23875000211522 USDC per 1 PHTR
Price Impact: -0.4999999059902449%
PHTR Price: 0.4444444444444444 USD
Total USD value of pool required to satisfy target_price_impact criterion: 9950001.88021535
Required USD to satisfy target yield: 995000.188021535
Required PHTR to satisfy target yield: 2238750.423048454


In [268]:
print('To summarise, the PHTR/USDC pool would need to have {x} PHTR and {y} USDC in order to produce less than {price_impact}% price impact on a {trade_size} USDC trade. In order to provide a {apr} APR on the {usd_pool_value} USD pool size we would need to distribute {phtr_to_distribute} PHTR (at the estimated PHTR price).'.format(x = x,y=y,price_impact = target_price_impact,apr = target_apr,usd_pool_value = usd_required,trade_size = target_investment_size,phtr_to_distribute = phtr_required))

To summarise, the PHTR/USDC pool would need to have 11193752.11524227 PHTR and 4975000.940107675 USDC in order to produce less than -0.5% price impact on a 25000 USDC trade. In order to provide a 0.1 APR on the 995000.188021535 USD pool size we would need to distribute 2238750.423048454 PHTR (at the estimated PHTR price).
